In [4]:
import requests
from sklearn.datasets import load_svmlight_file
import os
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
import numpy as np
import bz2

### DNA

In [6]:
urls_dict = {"train":"https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/multiclass/dna.scale.tr",
                "val":"https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/multiclass/dna.scale.val",
                "test":"https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/multiclass/dna.scale.t"}
x_total = np.array([]).reshape(0, 180)
y_total = np.array([]).reshape(0, 3)
for split_name, url in urls_dict.items():
    file_path = f"dna/dna_{split_name}_raw"
    x, y  = load_svmlight_file(file_path, n_features=180)
    x = np.asarray(x.todense(), dtype=np.float32)
    y = y.reshape(-1, 1).astype(np.float32)
    # pre-process the labels
    encoder = OneHotEncoder(sparse_output=False)
    y = encoder.fit_transform(y)
    x_total = np.vstack((x_total, x))
    y_total = np.vstack((y_total, y))

In [7]:
print(x_total.shape)
print(y_total.shape)

(3186, 180)
(3186, 3)


In [9]:
np.savez("dna/dna.npz", x=x_total, y=y_total)

In [10]:
with np.load("dna/dna.npz", allow_pickle=True) as file:
    x = file["x"].astype(np.float32)
    y = file["y"].astype(np.float32)

In [11]:
print(x.shape)
print(y.shape)

(3186, 180)
(3186, 3)


### Splice

In [2]:
urls_dict = {"train":"https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/binary/splice",
            "test": "https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/binary/splice.t"}

x_total = np.array([]).reshape(0, 60)
y_total = np.array([]).reshape(0, 2)
for split_name, url in urls_dict.items():
    file_path = f"splice/splice_{split_name}_raw"
    with open(file_path, 'w') as f:
        r = requests.get(url)
        f.writelines(r.content.decode("utf-8"))
    x, y  = load_svmlight_file(file_path, n_features=60)
    x = np.asarray(x.todense(), dtype=np.float32)
    y = y.reshape(-1, 1).astype(np.float32)
    # pre-process the labels
    encoder = OneHotEncoder(sparse_output=False)
    y = encoder.fit_transform(y)
    # pre-process the features
    scaler = MinMaxScaler()
    x = scaler.fit_transform(x)
    x_total = np.vstack((x_total, x))
    y_total = np.vstack((y_total, y))

In [3]:
print(x_total.shape)
print(y_total.shape)

(3175, 60)
(3175, 2)


In [4]:
np.savez("splice/splice.npz", x=x_total, y=y_total)

### Protein

In [15]:
urls_dict = {"t":"https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/multiclass/protein.tr.bz2",
            "tr": "https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/multiclass/protein.t.bz2",
            "val": "https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/multiclass/protein.val.bz2"}

x_total = np.array([]).reshape(0, 357)
y_total = np.array([]).reshape(0, 3)
for split_name, url in urls_dict.items():
    file_path = f"protein/protein.{split_name}"
    x, y  = load_svmlight_file(file_path, n_features=357)
    x = np.asarray(x.todense(), dtype=np.float32)
    y = y.reshape(-1, 1).astype(np.float32)
    # pre-process the labels
    encoder = OneHotEncoder(sparse_output=False)
    y = encoder.fit_transform(y)
    # pre-process the features
    scaler = MinMaxScaler()
    x = scaler.fit_transform(x)
    x_total = np.vstack((x_total, x))
    y_total = np.vstack((y_total, y))

In [16]:
print(x_total.shape)
print(y_total.shape)

(24387, 357)
(24387, 3)


In [17]:
np.savez("protein/protein.npz", x=x_total, y=y_total)

### Electicity dataset

In [19]:
df = pd.read_csv("electricalFault/detect_dataset.csv", header=0, index_col=None)
df.drop(columns=['Unnamed: 7', 'Unnamed: 8'], inplace=True)
df.rename(columns={'Output (S)': 'target'}, inplace=True)
scaler = MinMaxScaler()
df[df.columns.difference(['target'])] = scaler.fit_transform(df[df.columns.difference(['target'])])
np.savez("electricalFault/detect.npz", x=df[df.columns.difference(['target'])].values, y=df['target'].values)

### pokerdataset

In [21]:
df = pd.read_csv("pokerdataset/poker-hand-training.csv", header=0, index_col=None)
df.rename(columns={
    'Poker Hand': 'target'
}, inplace=True)
np.savez("pokerdataset/poker.npz", x=df[df.columns.difference(['target'])].values, y=df['target'].values)